In [0]:
%pip install databricks-vectorsearch
dbutils.library.restartPython()

In [0]:
# UC CATALOG, SCHEMA, INDEX
UC_CATALOG = "users"
UC_SCHEMA = "alex_miller"
VS_INDEX_NAME = "vs_batch_example"
SOURCE_TABLE = f"{UC_CATALOG}.{UC_SCHEMA}.imdb_embeddings"

# VS Endpoint Name
VECTOR_SEARCH_ENDPOINT = "abs_test_temp"
ENDPOINT_TYPE = "STORAGE_OPTIMIZED"  # or STANDARD

# Index-Name
VECTOR_SEARCH_INDEX = f"{UC_CATALOG}.{UC_SCHEMA}.{VS_INDEX_NAME}"

# Embedding Dimensions
EMBEDDING_DIMENSION = 1024

# Update sync 
update_index = False

In [0]:
from databricks.vector_search.client import VectorSearchClient

# The following line automatically generates a PAT Token for authentication
client = VectorSearchClient()

# Check if endpoint exists by iterating through the endpoints list
endpoints_response = client.list_endpoints()
endpoint_exists = any(
    endpoint['name'] == VECTOR_SEARCH_ENDPOINT 
    for endpoint in endpoints_response.get('endpoints', [])
)

if endpoint_exists:
    print("Endpoint already created.....")
else:
    print(f"Creating endpoint {VECTOR_SEARCH_ENDPOINT}")
    client.create_endpoint_and_wait(
        name=VECTOR_SEARCH_ENDPOINT,
        endpoint_type=ENDPOINT_TYPE
    )

In [0]:
# Check if index exists
indexes_response = client.list_indexes(VECTOR_SEARCH_ENDPOINT)
index_exists = any(
    index['name'] == VECTOR_SEARCH_INDEX 
    for index in indexes_response.get('vector_indexes', [])
)

if index_exists:
    print(f"{VECTOR_SEARCH_INDEX} already exists.....")
    if update_index:
        print(f"Updating index {VECTOR_SEARCH_INDEX}")
        index = client.get_index(index_name=VECTOR_SEARCH_INDEX)
        index.sync()
else:
    print(f"Creating index {VECTOR_SEARCH_INDEX}")
    index = client.create_delta_sync_index_and_wait(
        endpoint_name=VECTOR_SEARCH_ENDPOINT,
        source_table_name=SOURCE_TABLE,
        index_name=VECTOR_SEARCH_INDEX,
        pipeline_type="TRIGGERED",
        primary_key="id",
        embedding_dimension=1024,
        embedding_vector_column="embeddings"
    )